In [141]:
import ipykernel
ipykernel.__version__

import numpy as np
import pandas as pd
import networkx as nx
import scipy as sc
#from scipy import io
#from collections import Counter
from matplotlib import pyplot as plt
from signet.cluster import Cluster
from sklearn.model_selection import train_test_split
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import confusion_matrix
#from sklearn.metrics import plot_confusion_matrix
#from sklearn.metrics import plot_roc_curve
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split

In [142]:
df_small = pd.read_csv("Datasets/dataset_small.csv")
#df_small = pd.read_csv("test11.csv")
df_small = df_small.drop_duplicates(inplace = False)
df_small_p = df_small[df_small['sign'] == 1]
df_small_n = df_small[df_small['sign'] == -1]

In [143]:
df_small_train_p, df_small_test_p, df_small_train_label_p, df_small_test_label_p = train_test_split(df_small_p, df_small_p['source'], test_size = 0.3)
df_small_train_n, df_small_test_n, df_small_train_label_n, df_small_test_label_n = train_test_split(df_small_n, df_small_n['source'], test_size = 0.3)
df_small_train_final = pd.concat([df_small_train_p, df_small_train_n])
df_small_test_final = pd.concat([df_small_test_p, df_small_test_n])

In [144]:
df_small_test_final.head()

,source,target,sign
482,170,165,1
66,42,35,1
179,92,94,1
139,72,74,1
385,121,69,1


In [145]:
def relabel(df):
    source_old_labels = df["source"].drop_duplicates().to_numpy()
    #print(source_old_labels)
    target_old_labels = df["target"].drop_duplicates().to_numpy()
    all_old_labels = np.union1d(source_old_labels, target_old_labels)
    index_map = dict(enumerate(all_old_labels))
    new_index_map = dict([(value, key) for key, value in index_map.items()])
    #print(all_old_labels)
    new_df = df.copy()
    for index, row in new_df.iterrows():
        #print(row["source"])
        row["source"] = new_index_map[row["source"]]
        row["target"] = new_index_map[row["target"]]
    return new_index_map, new_df

In [146]:
ind_map, df_small_test_final = relabel(df_small_test_final)

In [147]:
df_small_test_final.head()

,source,target,sign
482,110,109,1
66,32,25,1
179,68,69,1
139,56,58,1
385,86,53,1


In [148]:
G = nx.DiGraph()
G = nx.from_pandas_edgelist(df = df_small_test_final, source = 'source', target = 'target', edge_attr = 'sign', create_using = nx.DiGraph(), edge_key = 'sign')

In [149]:
nodes_no = nx.number_of_nodes(G)
edges_no = nx.number_of_edges(G)

In [150]:
print('number of nodes G =',nx.number_of_nodes(G) )
print('number of edges G =',nx.number_of_edges(G) )
print('Density of G:', nx.density(G))

number of nodes G = 139
number of edges G = 177
Density of G: 0.009227400688145137


In [151]:
def getSimiliarity(G, sign_tag = 'sign'):
    nodes_no = nx.number_of_nodes(G)
    sign_map = nx.get_edge_attributes(G, sign_tag)
    # similiarity_matrix = np.zeros(shape = (nodes_no, nodes_no))
    pos_matrix = np.zeros(shape = (nodes_no, nodes_no))
    neg_matrix = np.zeros(shape = (nodes_no, nodes_no))
    for u in G.nodes():
        for v in G.nodes():
            #print("u: ", u, ", v: ", v)
            if (u != v):
                u_neighbors = set(G.adj[u])
                v_neighbors = set(G.adj[v])
                uv_neighbors = list(u_neighbors.intersection(v_neighbors))
                # similiarity_matrix[u - 1][v - 1] = np.sum([sign_map[(u, i)] * sign_map[(v, i)] for i in uv_neighbors])
                uv_similiarity = np.sum([sign_map[(u, i)] * sign_map[(v, i)] for i in uv_neighbors])
                if len(uv_neighbors) != 0:
                    uv_similiarity /= len(uv_neighbors)
                if (uv_similiarity > 0):
                    pos_matrix[u - 1][v - 1] = 1
                elif (uv_similiarity < 0):
                    neg_matrix[u - 1][v - 1] = 1
            #else:
                #similiarity_matrix[u - 1][v - 1] = 0
    pos_csc = sc.sparse.csc_matrix(pos_matrix)
    neg_csc = sc.sparse.csc_matrix(neg_matrix)
    return pos_csc, neg_csc


In [152]:
print(getSimiliarity(G, sign_tag = 'sign'))

(<139x139 sparse matrix of type '<class 'numpy.float64'>'
	with 126 stored elements in Compressed Sparse Column format>, <139x139 sparse matrix of type '<class 'numpy.float64'>'
	with 46 stored elements in Compressed Sparse Column format>)


In [153]:
#rows = df_small.loc[:, "source"].to_numpy()
#cols = df_small.loc[:, "target"].to_numpy()
#sign = df_small.loc[:, "sign"].to_numpy()

In [154]:
#p_rows = [rows[i] for i in range(len(rows)) if sign[i] == 1]
#p_cols = [cols[i] for i in range(len(cols)) if sign[i] == 1]
#p_data = np.ones(len(p_rows))

#n_rows = [rows[i] for i in range(len(rows)) if sign[i] == -1]
#n_cols = [cols[i] for i in range(len(cols)) if sign[i] == -1]
#n_data = np.ones(len(n_rows))

In [155]:
#A_p = sc.sparse.csc_matrix((p_data, (p_rows, p_cols)), shape = (131829,131829))
#A_n = sc.sparse.csc_matrix((n_data, (n_rows, n_cols)), shape = (131829,131829))
#A_p = A_p+A_p.T
#A_n = A_n+A_n.T
#print("A_p",A_p)
#print("A_n",A_n)

In [156]:
#print(p_data.shape)
#print(n_data.shape)

In [157]:
A_p, A_n = getSimiliarity(G, sign_tag = 'sign')

In [158]:
c = Cluster((A_p, A_n))

In [159]:
def getClusters(G, k):
    spec_clus = c.spectral_cluster_bnc(k = 2, normalisation = 'sym', eigens = None, mi = None)
    clusters = []
    for j in range(k):
        clusters.append([i for i in G.nodes() if spec_clus[i - 1] == j])
    return np.array(clusters, dtype = np.ndarray)

In [160]:
clusters = getClusters(G, 2)
cluster1 = clusters[0]
cluster2 = clusters[1]
#cluster3 = clusters[2]
#cluster4 = clusters[3]


print("cluster1:",cluster1)
print()
print("cluster2:",cluster2)
print()
#print("cluster3:",cluster3)
#print()
#print("cluster4:",cluster4)
#print()

cluster1: [110, 109, 32, 25, 68, 69, 56, 58, 86, 53, 130, 135, 42, 100, 67, 8, 17, 99, 127, 133, 78, 41, 131, 66, 95, 33, 19, 73, 74, 122, 97, 126, 124, 113, 96, 51, 60, 64, 61, 48, 91, 89, 3, 123, 47, 35, 71, 72, 38, 107, 102, 128, 6, 92, 54, 4, 134, 52, 40, 82, 28, 10, 16, 39, 103, 21, 15, 24, 132, 55, 79, 1, 138, 14, 83, 2, 70, 105, 125, 81, 44, 49, 111, 23, 20, 88, 106, 84, 18, 108, 43, 34, 115, 120, 80, 12, 112, 137, 31, 59, 36, 30, 27, 117, 57, 101, 45, 119, 11, 104, 22, 114, 93, 85, 26, 29, 65, 77, 37, 76, 136, 121, 129, 75, 9, 0, 62, 116, 98, 90, 94, 87, 5, 50, 46, 7, 118]

cluster2: [13, 63]



C:\Users\Alireza\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [161]:
print(len(cluster1))
print(len(cluster2))
#print(len(cluster3))
#print(len(cluster4))


137
2


In [162]:
def getCommonNeighbors(G, cl1, cl2):
    x = set([])
    for i in cl1:
        x = x.union(set(G.adj[i]))
    #print('x: ', x)
    y = set([])
    for j in cl2:
        y = y.union(set(G.adj[j]))
    #print('y: ', y)
    return list(x.intersection(y))

def getClusterSimiliarity(Cluster, common_neighbors, sign_tag = 'sign'):
    sign_map = nx.get_edge_attributes(G, sign_tag)
    y = []
    for v in common_neighbors:
        x = []
        for u in Cluster:
            if (u, v) in G.edges():
                x.append(sign_map[(u, v)])
        y.append(np.mean(np.array(x)))
    return np.array(y)


def getSimiliarity(y1, y2):
    alpha = np.dot(y1.T, y2)
    beta = np.dot(y1.T, y1)
    gamma = np.dot(y2.T, y2)
    return alpha / np.sqrt((beta * gamma))


# main function to calculate the similiarity between two clusters
def getInterClusterSimiliarity(cl1, cl2, common_neighbors):
    y1 = getClusterSimiliarity(cl1, common_neighbors)
    y2 = getClusterSimiliarity(cl2, common_neighbors)
    return getSimiliarity(y1, y2)

# calculates similiarities between each two clusters and returns a matrix
def getAllSimiliaritiesBetweenClusters(G, clusters):
    clusters_no = clusters.shape[0]
    similiarities = np.zeros(shape = (clusters_no, clusters_no))
    for i in range(clusters_no):
        for j in range(i, clusters_no):
            common_neighbors = getCommonNeighbors(G, clusters[i], clusters[j])
            similiarities[i][j] = similiarities[j][i] = getInterClusterSimiliarity(clusters[i], clusters[j], common_neighbors)
    return similiarities

In [163]:
common_neighbors = getCommonNeighbors(G, cluster1, cluster2)
common_neighbors
y1 = getClusterSimiliarity(cluster1, common_neighbors)
y2 = getClusterSimiliarity(cluster2, common_neighbors)
similarity = getInterClusterSimiliarity(cluster1, cluster2, common_neighbors)
similarity 
matrix_of_similarity = getAllSimiliaritiesBetweenClusters(G, clusters)
matrix_of_similarity

array([[ 1., -1.],
       [-1.,  1.]])

In [164]:
def getNeighborsOfANode(G, u):
    x = []
    for v in G.nodes():
        if ((v, u) in G.edges()):
            x.append(v)
    return np.array(x)

In [165]:
def locationOfANode(clusters, u):
    n = clusters.shape[0]
    x = np.zeros(n)
    r = -1
    for i in range(n):
        if u in clusters[i]:
            r = i
    return r

def locationOfArrayOfNode(clusters, U):
    x = np.zeros(len(U))
    for i in range(len(x)):
        x[i] = locationOfANode(clusters, U[i])
    return x

In [166]:
def signPrediction(G, k, u, v, threshold, sign_tag = "sign"):
    AA1 = nx.get_edge_attributes(G, sign_tag)
    main_clusters = getClusters(G, k)
    u_c = locationOfANode(main_clusters, u)
    u_v = locationOfANode(main_clusters, v)
    cl1 = main_clusters[u_c]
    cl2 = main_clusters[u_v]
    s_AB = getCommonNeighbors(G, cl1, cl2)
    #print(s_AB)
    S = getNeighborsOfANode(G, v)
    S_c = locationOfArrayOfNode(main_clusters, S)
    n = len(S)
    x = np.zeros(n)
    for i in range(n):
        x[i] = getInterClusterSimiliarity(main_clusters[u_c], main_clusters[int(S_c[i])], s_AB)
    r = np.zeros(n)
    for i in range(n):
        r[i] = AA1[(S[i], v)]
    #print(x)
    #print(r)
    result = (np.dot(x.T, r)) / (np.sum(x))
    if result > threshold:
        return 1
    else:
        return -1

In [167]:
zz = np.zeros(df_small_test_final.shape[0])
df_small_test_final['sign prediction'] = df_small_test_final.apply(lambda row : signPrediction(G, 2, row['source'], row['target'], 0), axis = 1)
confusion_matrix(df_small_test_final['sign'], df_small_test_final['sign prediction'])
accuracy_score(df_small_test_final['sign'], df_small_test_final['sign prediction'],  normalize=True, sample_weight=None)

C:\Users\Alireza\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\Alireza\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\Alireza\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\U

0.9378531073446328